# 그래프 뉴럴 네트워크를 사용한 링크 예측

GNN은 그래프 데이터에 대한 많은 머신러닝 task를 해결하는 데 강력한 툴입니다.  
이 튜토리얼에서는, 링크 예측을 위해 GNN을 사용하는 기본적인 워크플로우를 배울 수 있습니다.  
여기서 Zachery의 카라테 클럽 그래프를 다시 사용합니다. 하지만, 이번에는 두 멤버 사이의 관계를 예측하는 작업을 시도해 봅니다.

이 튜토리얼에서, 다음을 배울 수 있습니다.
* 링크 예측을 위한 학습/테스트 셋을 준비하는 방법
* GNN 기반 링크 예측 모델을 구축하는 법
* 모델을 학습하고, 그 결과를 검증하는 법


In [1]:
import dgl
import torch
import torch.nn as nn
import torch.nn.functional as F
import itertools
import numpy as np
import pandas as pd
import scipy.sparse as sp


## 그래프와 피처 로드

최근 튜토리얼 [세션](./3_gnn.ipynb)에 이어, Zachery의 카라테 클럽 그래프를 불러들여 노드 임베딩을 만듭니다.

In [2]:
def load_zachery():
    nodes_data = pd.read_csv('../data/graph_data/dgl_nodes.csv')
    edges_data = pd.read_csv('../data/graph_data/dgl_edges.csv')
    src = edges_data['Src'].to_numpy()
    dst = edges_data['Dst'].to_numpy()
    g = dgl.graph((src, dst))
    club = nodes_data['Club'].to_list()
    # Convert to categorical integer values with 0 for 'Mr. Hi', 1 for 'Officer'.
    club = torch.tensor([c == 'Officer' for c in club]).long()
    # We can also convert it to one-hot encoding.
    club_onehot = F.one_hot(club)
    g.ndata.update({'club': club, 'club_onehot': club_onehot})
    return g

In [3]:
# ----------- 0. load graph -------------- #
g = load_zachery()
print(g)

# ----------- 1. node features -------------- #
node_embed = nn.Embedding(g.number_of_nodes(), 5)  # 각 노드는 5차원의 임베딩을 가지고 있습니다.
inputs = node_embed.weight                         # 노드 피처로써 이 임베딩 가중치를 사용합니다.
nn.init.xavier_uniform_(inputs)

Graph(num_nodes=34, num_edges=156,
      ndata_schemes={'club': Scheme(shape=(), dtype=torch.int64), 'club_onehot': Scheme(shape=(2,), dtype=torch.int64)}
      edata_schemes={})


Parameter containing:
tensor([[ 0.1645,  0.0892, -0.0433,  0.1239,  0.0345],
        [ 0.1802,  0.1879,  0.1859, -0.3832,  0.1308],
        [ 0.0643,  0.1033, -0.2554, -0.2468, -0.1306],
        [ 0.0871, -0.1111,  0.0621, -0.2267,  0.1116],
        [-0.2492,  0.1231, -0.3192, -0.3357,  0.1718],
        [ 0.1839,  0.3496,  0.0867,  0.1509, -0.0055],
        [-0.3257, -0.2141, -0.3664,  0.1968, -0.0072],
        [-0.1056,  0.3131, -0.0744,  0.2514,  0.2938],
        [ 0.0674, -0.0395, -0.1452,  0.0791, -0.0766],
        [-0.3083,  0.1055, -0.2634,  0.2780,  0.3064],
        [ 0.3188, -0.2478,  0.1480, -0.2389,  0.1614],
        [ 0.3389, -0.0906, -0.2838, -0.3307, -0.0066],
        [ 0.3858,  0.1162,  0.1649,  0.3192,  0.3624],
        [ 0.2681,  0.0290,  0.2492, -0.1742,  0.1582],
        [-0.1309, -0.2332, -0.1294, -0.1640,  0.3085],
        [ 0.2250, -0.0612,  0.2069,  0.3304, -0.0375],
        [ 0.0935,  0.3383, -0.1534, -0.3894,  0.3328],
        [ 0.0650,  0.1148,  0.3669, -0.2740

## 학습/테스트 셋을 준비 합니다

일반적으로, 링크 예측 데이터셋은 *positive*와 *negative* 엣지라는 2 타입의 엣지를 포함하고 있습니다.  
positive 엣지는 보통 그래프 내에 이미 존재하는 엣지로부터 가져옵니다.  
이 예제에서, 50개의 임의의 엣지를 골라 테스트에 사용하고 나머지는 학습에 사용합니다.

In [5]:
# 학습과 테스트를 위해 엣지 셋을 분할합니다.
u, v = g.edges()
eids = np.arange(g.number_of_edges())
eids = np.random.permutation(eids)
test_pos_u, test_pos_v = u[eids[:50]], v[eids[:50]]
train_pos_u, train_pos_v = u[eids[50:]], v[eids[50:]]

negative 엣지의 수가 크기때문에, 보통 샘플링 해주는 것이 좋습니다.   
적절한 negative 샘플링 알고리즘을 선택하는 방법에 대한 문제는 널리 연구되는 주제로, 이 튜토리얼의 범위를 벗어납니다.  
우리의 예제 그래프는 상당히 작기 때문에(노드 34개뿐), 모든 결측 엣지를 나열해 임의로 50개를 테스트에 사용하고, 150개를 학습에 사용합니다.


In [6]:
# 모든 negative 엣지를 찾아 학습과 테스트용으로 분할
adj = sp.coo_matrix((np.ones(len(u)), (u.numpy(), v.numpy())))
adj_neg = 1 - adj.todense() - np.eye(34)
neg_u, neg_v = np.where(adj_neg != 0)
neg_eids = np.random.choice(len(neg_u), 200)
test_neg_u, test_neg_v = neg_u[neg_eids[:50]], neg_v[neg_eids[:50]]
train_neg_u, train_neg_v = neg_u[neg_eids[50:]], neg_v[neg_eids[50:]]

Put positive and negative edges together and form training and testing sets.

In [7]:
# Create training set.
train_u = torch.cat([torch.as_tensor(train_pos_u), torch.as_tensor(train_neg_u)])
train_v = torch.cat([torch.as_tensor(train_pos_v), torch.as_tensor(train_neg_v)])
train_label = torch.cat([torch.ones(len(train_pos_u)), torch.zeros(len(train_neg_u))])

# Create testing set.
test_u = torch.cat([torch.as_tensor(test_pos_u), torch.as_tensor(test_neg_u)])
test_v = torch.cat([torch.as_tensor(test_pos_v), torch.as_tensor(test_neg_v)])
test_label = torch.cat([torch.ones(len(test_pos_u)), torch.zeros(len(test_neg_u))])

## GraphSAGE 모델 정의하기

우리의 모델은 2개 레이어로 구성되어 있는데, 각각 새로운 노드 표현(representation)을 이웃의 정보를 통합함으로써 계산합니다.  
수식은 다음과 같습니다.  ([이전 튜토리얼](./3_gnn.ipynb)과 약간 다릅니다.)

$$
h_{\mathcal{N}(v)}^k\leftarrow \text{AGGREGATE}_k\{h_u^{k-1},\forall u\in\mathcal{N}(v)\}
$$

$$
h_v^k\leftarrow \text{ReLU}\left(W^k\cdot \text{CONCAT}(h_v^{k-1}, h_{\mathcal{N}(v)}^k) \right)
$$

DGL은 많은 유명한 이웃 통합(neighbor aggregation) 모듈의 구현체를 제공합니다. 모두 쉽게 한 줄의 코드로 호출하여 사용할 수 있습니다.  
지원되는 모델의 전체 리스트는 [graph convolution modules](https://docs.dgl.ai/api/python/nn.pytorch.html#module-dgl.nn.pytorch.conv)에서 보실 수 있습니다.

In [8]:
from dgl.nn import SAGEConv

# ----------- 2. create model -------------- #
# 2개의 레이어를 가진 GraphSAGE 모델 구축
class GraphSAGE(nn.Module):
    def __init__(self, in_feats, h_feats):
        super(GraphSAGE, self).__init__()
        self.conv1 = SAGEConv(in_feats, h_feats, 'mean')
        self.conv2 = SAGEConv(h_feats, h_feats, 'mean')
    
    def forward(self, g, in_feat):
        h = self.conv1(g, in_feat)
        h = F.relu(h)
        h = self.conv2(g, h)
        return h

# 주어진 차원의 모델 생성
# 인풋 레이어 차원: 5, 노드 임베딩
# 히든 레이어 차원: 16
net = GraphSAGE(5, 16)

그 뒤, 모델을 아래의 손실함수를 사용해 최적화합니다.

$$
\hat{y}_{u\sim v} = \sigma(h_u^T h_v)
$$

$$
\mathcal{L} = -\sum_{u\sim v\in \mathcal{D}}\left( y_{u\sim v}\log(\hat{y}_{u\sim v}) + (1-y_{u\sim v})\log(1-\hat{y}_{u\sim v})) \right)
$$

기본적으로, 모델은 엣지의 두 끝지점(노드)의 표현을 내적함으로써 각 엣지에 대한 점수를 계산합니다.  
그 뒤 타겟 y가 0 혹은 1인 binary cross entropy loss를 계산합니다. 여기서 0 혹은 1은 해당 엣지가 positive인지 아닌지를 나타냅니다.


In [9]:
# ----------- 3. set up loss and optimizer -------------- #
# 이 경우, 학습 루프의 손실
optimizer = torch.optim.Adam(itertools.chain(net.parameters(), node_embed.parameters()), lr=0.01)

# ----------- 4. training -------------------------------- #
all_logits = []
for e in range(100):
    # forward
    logits = net(g, inputs)
    pred = torch.sigmoid((logits[train_u] * logits[train_v]).sum(dim=1))

    # 손실 계산
    loss = F.binary_cross_entropy(pred, train_label)

    # backward
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    all_logits.append(logits.detach())

    if e % 5 == 0:
        print('In epoch {}, loss: {}'.format(e, loss))

In epoch 0, loss: 2.93039608001709
In epoch 5, loss: 0.8791418075561523
In epoch 10, loss: 0.6970135569572449
In epoch 15, loss: 0.6595255136489868
In epoch 20, loss: 0.6340247392654419
In epoch 25, loss: 0.6087173819541931
In epoch 30, loss: 0.585627555847168
In epoch 35, loss: 0.5588462948799133
In epoch 40, loss: 0.5267122983932495
In epoch 45, loss: 0.49372491240501404
In epoch 50, loss: 0.4604291021823883
In epoch 55, loss: 0.42443352937698364
In epoch 60, loss: 0.38596004247665405
In epoch 65, loss: 0.3501429557800293
In epoch 70, loss: 0.31591594219207764
In epoch 75, loss: 0.28520211577415466
In epoch 80, loss: 0.25504064559936523
In epoch 85, loss: 0.2231801450252533
In epoch 90, loss: 0.19235175848007202
In epoch 95, loss: 0.16256636381149292


In [10]:
# ----------- 5. check results ------------------------ #
pred = torch.sigmoid((logits[test_u] * logits[test_v]).sum(dim=1))
print('Accuracy', ((pred >= 0.5) == test_label).sum().item() / len(pred))

Accuracy 0.77
